In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing Functions

In [ ]:
!pip install fuzzywuzzy -q

In [ ]:
import re
import pandas as pd
import nltk
import json
from nltk.tokenize import word_tokenize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s,.*=<>()]', '', text)
    return text

def tokenize_text(text):
    return word_tokenize(text)

def parse_schema(schema):
    tables = {}
    for table_idx, table_name in enumerate(schema['table_names']):
        columns = [schema['column_names'][idx][1] for idx in range(len(schema['column_names'])) if schema['column_names'][idx][0] == table_idx]
        tables[table_name] = {'columns': columns}
    return tables

def link_question_to_schema(question, tables):
    linked_elements = {}
    tokens = tokenize_text(question)
    for token in tokens:
        best_match = None
        highest_score = 0
        for table_name, info in tables.items():
            # Check table name match
            table_score = fuzz.partial_ratio(token.lower(), table_name.lower())
            if table_score > highest_score:
                best_match = ('table', table_name)
                highest_score = table_score
            # Check column name match
            for column in info['columns']:
                column_score = fuzz.partial_ratio(token.lower(), column.lower())
                if column_score > highest_score:
                    best_match = ('column', column)
                    highest_score = column_score
        if highest_score > 80:  # Adjust the threshold as needed
            linked_elements[token] = best_match
    return linked_elements


In [ ]:
def preprocess_data(data, tables):
    """ Preprocess data by normalizing, tokenizing, linking to schema, and extracting features """
    processed_data = []
    questions = []
    schema_links = []
    sql_queries = []

    for entry in data:
        question = entry['question']
        normalized_question = normalize_text(question)
        tokenized_question = tokenize_text(normalized_question)
        schema_linked = link_question_to_schema(normalized_question, tables)

        processed_data.append({
            'original_question': question,
            'normalized_question': normalized_question,
            'tokenized_question': tokenized_question,
            'schema_links': schema_linked,
            'sql_query': entry['query']
        })

        questions.append(question)
        schema_links.append(schema_linked)
        sql_queries.append(entry['query'])

    return processed_data, questions, schema_links, sql_queries

In [ ]:
def preprocess_all_data(train_data, db_schemas):
    all_processed_data = []
    questions_all = []
    schema_links_all = []
    sql_queries_all = []

    schema_mapping = {schema['db_id']: parse_schema(schema) for schema in db_schemas}

    for entry in train_data:
        db_id = entry['db_id']
        schema = schema_mapping[db_id]
        processed_entry, questions, schema_links, sql_queries = preprocess_data([entry], schema)
        all_processed_data.extend(processed_entry)

        questions_all.extend(questions)
        schema_links_all.extend(schema_links)
        sql_queries_all.extend(sql_queries)

     # Create a DataFrame from accumulated lists
    df = pd.DataFrame({
        'question': questions_all,
        'schema_links': schema_links_all,
        'sql_query': sql_queries_all
    })

    return all_processed_data, df


### Loading JSON Data and Preprocessing It

In [ ]:
import json

In [ ]:
def load_json_file(file_path):
    """ Load a JSON file and return the data """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [ ]:
db_schemas = load_json_file('/content/drive/MyDrive/ Text to Sql/spider/tables.json')
train_data = load_json_file('/content/drive/MyDrive/ Text to Sql/spider/train_spider.json')

In [ ]:
processed_train_data, df_preprocessed_training_data_all = preprocess_all_data(train_data, db_schemas)

In [ ]:
processed_train_data

[{'original_question': 'How many heads of the departments are older than 56 ?',
  'normalized_question': 'how many heads of the departments are older than 56 ',
  'tokenized_question': ['how',
   'many',
   'heads',
   'of',
   'the',
   'departments',
   'are',
   'older',
   'than',
   '56'],
  'schema_links': {'heads': ('table', 'head'),
   'departments': ('table', 'department')},
  'sql_query': 'SELECT count(*) FROM head WHERE age  >  56'},
 {'original_question': 'List the name, born state and age of the heads of departments ordered by age.',
  'normalized_question': 'list the name, born state and age of the heads of departments ordered by age.',
  'tokenized_question': ['list',
   'the',
   'name',
   ',',
   'born',
   'state',
   'and',
   'age',
   'of',
   'the',
   'heads',
   'of',
   'departments',
   'ordered',
   'by',
   'age',
   '.'],
  'schema_links': {'name': ('column', 'name'),
   'born': ('column', 'born state'),
   'state': ('column', 'born state'),
   'age': ('co

In [ ]:
df_preprocessed_training_data_all

,question,schema_links,sql_query
0,How many heads of the departments are older th...,"{'heads': ('table', 'head'), 'departments': ('...",SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","{'name': ('column', 'name'), 'born': ('column'...","SELECT name , born_state , age FROM head ORD..."
2,"List the creation year, name and budget of eac...","{'creation': ('column', 'creation'), 'name': (...","SELECT creation , name , budget_in_billions ..."
3,What are the maximum and minimum budget of the...,"{'budget': ('column', 'budget in billions'), '...","SELECT max(budget_in_billions) , min(budget_i..."
4,What is the average number of employees of the...,"{'average': ('column', 'age'), 'employees': ('...",SELECT avg(num_employees) FROM department WHER...
...,...,...,...
6995,What are all the company names that have a boo...,"{'are': ('column', 'group equity shareholding'...",SELECT T1.company_name FROM culture_company AS...
6996,Show the movie titles and book titles for all ...,"{'movie': ('table', 'movie'), 'titles': ('colu...","SELECT T1.title , T3.book_title FROM movie AS..."
6997,What are the titles of movies and books corres...,"{'are': ('column', 'group equity shareholding'...","SELECT T1.title , T3.book_title FROM movie AS..."
6998,Show all company names with a movie directed i...,"{'company': ('table', 'culture company'), 'nam...",SELECT T2.company_name FROM movie AS T1 JOIN c...


### Model Architecture

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
class TextToSQLModel(nn.Module):
    def __init__(self, tokenizer_vocab_size):
        super(TextToSQLModel, self).__init__()
        self.encoder = BertModel.from_pretrained('bert-base-uncased')
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=768, nhead=8), num_layers=6
        )
        self.embedding = nn.Embedding(tokenizer_vocab_size, 768)
        self.output_linear = nn.Linear(768, tokenizer_vocab_size)

    def forward(self, input_ids, attention_mask, tgt_tokens):
        encoder_hidden_states = self.encoder(input_ids=input_ids, attention_mask=attention_mask)[0]
        tgt_embeddings = self.embedding(tgt_tokens)  # Embedding target tokens
        decoder_outputs = self.decoder(tgt_embeddings, encoder_hidden_states)
        output = self.output_linear(decoder_outputs)
        return output

In [ ]:
class SQLDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        inputs = self.tokenizer.encode_plus(
            item['normalized_question'],
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        targets = self.tokenizer.encode(
            item['sql_query'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True
        )
        return {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'labels': torch.tensor(targets)
        }

In [ ]:
from tqdm import tqdm

def train_model(model, data_loader, optimizer, device, epochs=3):
  model.train()
  total_loss = 0
  for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(data_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False)
        for batch in progress_bar:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, labels)
            loss = nn.CrossEntropyLoss()(outputs.transpose(1, 2), labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'Batch Loss': loss.item()})

        avg_loss = total_loss / len(data_loader)
        total_loss += avg_loss
        print (f'End of Epoch {epoch+1}, Average Loss: {avg_loss:.4f}')
  return f"total loss : {total_loss} , total average loss ={total_loss/epochs}"

In [ ]:
def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, labels)
            loss = nn.CrossEntropyLoss()(outputs.transpose(1, 2), labels)
            total_loss += loss.item()
    return total_loss / len(data_loader)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Obtain the vocabulary size from the tokenizer
tokenizer_vocab_size = tokenizer.vocab_size

# Initialize the model with the required vocabulary size
model = TextToSQLModel(tokenizer_vocab_size).to(device)

# Set up the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
train_dataset = SQLDataset(processed_train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
# Call training
train_loss = train_model(model, train_loader, optimizer, device)

End of Epoch 1, Average Loss: 0.1399


End of Epoch 2, Average Loss: 0.0190


End of Epoch 3, Average Loss: 0.0080


In [ ]:
print("Training loss:", train_loss)

Training loss: total loss : 6.980382935216917 , total average loss =2.3267943117389724


### Save the Model

In [ ]:
def save_model(model, optimizer, file_path):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, file_path)

# Example usage:
save_path = '/content/drive/MyDrive/text_to_sql_model_full_trained.pth'
save_model(model, optimizer, save_path)

### Using the Model

In [ ]:
# Mock-up of schema parsing (assuming it's already defined)
schema = {
    'table_names': ['products', 'categories'],
    'column_names': [
        (0, 'product_id'), (0, 'name'), (0, 'price'), (0, 'category_id'),
        (1, 'category_id'), (1, 'category_name')
    ]
}

def parse_schema(schema):
    """ Parse the database schema into a dictionary mapping table names to their columns. """
    tables = {}
    for table_idx, table_name in enumerate(schema['table_names']):
        columns = [schema['column_names'][idx][1] for idx in range(len(schema['column_names'])) if schema['column_names'][idx][0] == table_idx]
        tables[table_name] = {'columns': columns}
    return tables

# Preprocess the input text
def preprocess_input(question, schema):
    normalized_question = normalize_text(question)
    tokenized_question = tokenize_text(normalized_question)
    schema_links = link_question_to_schema(normalized_question, schema)
    return normalized_question, schema_links

# Generate SQL Query from the model
def generate_sql_query(model, tokenizer, question, schema, device):
    normalized_question, schema_links = preprocess_input(question, schema)
    input_ids, attention_mask = prepare_model_input(normalized_question, tokenizer, schema_links, device)
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = outputs.argmax(dim=2)
    decoded_query = tokenizer.decode(predictions[0], skip_special_tokens=True)
    return decoded_query

# Example usage
question = "What are the names and prices of all products in the electronics category?"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sql_query = generate_sql_query(model, tokenizer, question, schema, device)
print("Generated SQL Query:", sql_query)


AttributeError: 'list' object has no attribute 'items'

In [ ]:
import re
import torch
from transformers import BertTokenizer
from nltk.tokenize import word_tokenize
from fuzzywuzzy import fuzz
import nltk
nltk.download('punkt')

# Define the schema
schema = {
    'table_names': ['products', 'categories'],
    'column_names': [
        (0, 'product_id'), (0, 'name'), (0, 'price'), (0, 'category_id'),
        (1, 'category_id'), (1, 'category_name')
    ]
}

# Preprocess functions
def normalize_text(text):
    """ Normalize text by lowering case and removing unwanted characters """
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s,.*=<>()]', '', text)
    return text

def tokenize_text(text):
    """ Tokenize text using nltk's word_tokenize """
    return word_tokenize(text)

# def parse_schema(schema):
#     """ Parse the database schema into a dictionary mapping table names to their columns """
#     tables = {}
#     for table_idx, table_name in enumerate(schema['table_names']):
#         # Make sure to extract only the column name as a string, not a tuple
#         columns = [schema['column_names'][idx][1] for idx in range(len(schema['column_names'])) if schema['column_names'][idx][0] == table_idx]
#         tables[table_name] = columns
#     return tables

def parse_schema(schema):
    """ Parse the database schema into a dictionary mapping table names to their columns """
    tables = {}
    for table_idx, table_name in enumerate(schema['table_names']):
        # Extract only the column name as a string, not a tuple
        columns = [schema['column_names'][idx][1] for idx in range(len(schema['column_names'])) if schema['column_names'][idx][0] == table_idx]
        tables[table_name] = columns
    return tables

def link_question_to_schema(question, tables):
    """ Link tokens from the question to schema elements using fuzzy matching """
    linked_elements = {}
    tokens = tokenize_text(question)
    for token in tokens:
        best_match = None
        highest_score = 0
        for table_name, columns in tables.items():
            # Check table name match
            table_score = fuzz.partial_ratio(token.lower(), table_name.lower())
            if table_score > highest_score:
                best_match = ('table', table_name)
                highest_score = table_score
            # Check column name match, ensuring column is a string
            for column in columns:
                # print(f"Comparing token {token} with column {column} (type: {type(column)})")
                column = column[1]
                # print(f"Comparing token {token} with column {column} (type: {type(column)})")
                column_score = fuzz.partial_ratio(token.lower(), column.lower())  # column must be a string
                if column_score > highest_score:
                    best_match = ('column', column)
                    highest_score = column_score
        if highest_score > 80:  # Adjust the threshold as needed
            linked_elements[token] = best_match
    return linked_elements

def generate_sql_query(model, tokenizer, question, schema, device):
    model.eval()

    # Normalize and preprocess the question
    normalized_question, schema_links = preprocess_input(question, schema)
    input_ids, attention_mask = prepare_model_input(normalized_question, tokenizer, schema_links, device)

    # Initialize target tokens with the start token
    start_token_id = tokenizer.cls_token_id
    tgt_tokens = torch.full((input_ids.size(0), 1), start_token_id, dtype=torch.long, device=device)

    # Match tgt_tokens length to input_ids length
    if tgt_tokens.size(1) < input_ids.size(1):
        padding = torch.full((input_ids.size(0), input_ids.size(1) - tgt_tokens.size(1)), tokenizer.pad_token_id, dtype=torch.long, device=device)
        tgt_tokens = torch.cat([tgt_tokens, padding], dim=1)

    print("Input IDs shape:", input_ids.shape)
    print("Target Tokens shape:", tgt_tokens.shape)
    print("Attention Mask shape:", attention_mask.shape)

    outputs = model(input_ids, attention_mask, tgt_tokens)  # Pass tgt_tokens matching the length of input_ids
    predictions = torch.argmax(outputs[:, -1, :], dim=-1)  # Getting the last token predictions
    decoded_query = tokenizer.decode(predictions, skip_special_tokens=True)
    return decoded_query

def beam_search(model, tokenizer, question, schema, device, beam_width=3):
    model.eval()
    # Initial setup
    input_ids, attention_mask = prepare_model_input(question, tokenizer, schema, device)
    start_token_id = tokenizer.cls_token_id
    sequences = [[start_token_id]]

    # Beam search
    for _ in range(50):  # Max sequence length
        all_candidates = []
        for seq in sequences:
            tgt_tokens = torch.tensor([seq], dtype=torch.long, device=device)
            with torch.no_grad():
                outputs = model(input_ids, attention_mask, tgt_tokens)
                next_token_logits = outputs[:, -1, :]
                probabilities = torch.softmax(next_token_logits, dim=-1)
            top_k_probs, top_k_tokens = torch.topk(probabilities, beam_width, dim=-1)

            # Expand each current candidate
            for i in range(beam_width):
                next_seq = seq + [top_k_tokens[0][i].item()]
                next_prob = top_k_probs[0][i].item()
                candidate = (next_seq, next_prob)
                all_candidates.append(candidate)

        # Order all candidates by probability
        ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
        sequences = [x[0] for x in ordered[:beam_width]]  # Narrow down to the best beam_width sequences

    print("Input IDs shape:", input_ids.shape)
    print("Target Tokens shape:", tgt_tokens.shape)
    print("Attention Mask shape:", attention_mask.shape)

    # Decode the best sequence
    best_seq = sequences[0]
    decoded_query = tokenizer.decode(best_seq, skip_special_tokens=True)
    return decoded_query


def prepare_model_input(question, tokenizer, schema_links, device):
    """ Prepare model input combining question and schema information """
    schema_info = ' '.join([f"{k}:{v}" for k, v in schema_links.items()])
    combined_input = f"{question} [SEP] {schema_info}"
    inputs = tokenizer.encode_plus(
        combined_input,
        None,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
        truncation=True
    )
    input_ids = torch.tensor(inputs['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(inputs['attention_mask']).unsqueeze(0).to(device)
    return input_ids, attention_mask


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
question = "What are the names and prices of all products in the electronics category?"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TextToSQLModel(tokenizer.vocab_size).to(device)  # Assume this is properly loaded

sql_query = beam_search(model, tokenizer, question, schema, device)

print("Generated SQL Query:", sql_query)

RuntimeError: shape '[1, 8, 96]' is invalid for input of size 393216

In [ ]:
sql_query

'[unused519]'

In [ ]:
question = "What are the names of all products in the electronics category?"
sql_query = generate_sql_query(model, tokenizer, question, schema, device)
print("Generated SQL Query:", sql_query)

Input IDs shape: torch.Size([1, 512])
Target Tokens shape: torch.Size([1, 512])
Attention Mask shape: torch.Size([1, 512])
Generated SQL Query: reassigned
